In [1]:
import os
import numpy as np
import regex as re
import pandas as pd

In [2]:
from collections import Counter

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package punkt to /home/varsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [4]:
from rapidfuzz import process as proc
from rapidfuzz import fuzz

In [5]:
from tqdm import tqdm

In [6]:
from fastparquet import ParquetFile, write as parq_write

In [7]:
import re
import time
from sentence_transformers import SentenceTransformer, util

/home/varsey/PycharmProjects/drl-case/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
import random

In [9]:
import yake

In [10]:
flt_chars = [
    ')', '(', ',', '!', '', ':', '|', '>', '<', '[', ']', '...', './', '/\n', '\xa0',
    '«', '»', '_', '?', '~', '=', '\\', ';', '\n', '``', "''", '✅', '/n', '/', '..', '`', "'", '-',
    '✔', '⇒', '"', '$', '%', '&', "'", ',', '^', '`', '{', '}', "('",
]

In [615]:
stop_words = (nltk.corpus.stopwords.words('russian')
              + nltk.corpus.stopwords.words('english')
              + nltk.corpus.stopwords.words('azerbaijani'))
exclude = [x for x in set(stop_words + flt_chars) if len(x) > 0] + ['pet']
exclude.remove('up')
exclude.remove('s')
exclude.remove('ya')
exclude = set(exclude)

In [592]:
'ya' in exclude

False

In [593]:
remove_chars = {
    '-': ' ',
    "'": ' ',
    "(": ' ',
    ")": ' ',
    '"': ' ',
    '<': ' ',
    '>': ' ',
    "/": ' ',
}
replace_chars = {
    '*': ' x ',
    "%": 'pct',
    ",": '.',
    " s ":  's '
}


In [594]:
def simple_process_item(x: str):
    x = x.lower()
    for char in remove_chars.keys():
        x = x.replace(char, ' ')
    for k, v in replace_chars.items():
        x = x.replace(k, v)
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    item = item.replace(' .', '.').replace('. ', '.')
    return ' '.join([x for x in item.split() if x not in exclude])

In [563]:
def count_common_words(str1, str2):
    set1 = set([x for x in str1.lower().split() if len(x) > 3 and not x.isdigit()])
    set2 = set([x for x in str2.lower().split() if len(x) > 3 and not x.isdigit()])
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [564]:
def count_common_digits(str1, str2):
    set1 = set([x for x in str1.lower().split() if x.isdigit()])
    set2 = set([x for x in str2.lower().split() if x.isdigit()])
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [565]:
def count_digit_share(x: str):
    counter = 0
    for char in x:
        if char.isdigit():
            counter += 1
    return counter/len(x.replace(' ', '')) if len(x) > len(x.replace(' ', '')) and counter > 0 else 0.5

In [566]:
def remove_numbers(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split('.'))
    return ' '.join([x for x in text.split() if len(x) > 2 and x not in ('pct', 'dad')][:3])

In [567]:
# Loading data
df_incoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory Out')
df_outcoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory In')

In [568]:
df_outcoming['unit'] = df_incoming['Unit'].str.lower()
df_incoming['unit'] = df_incoming['Unit'].str.lower()

In [569]:
text = df_outcoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

Mimi Yoqurt Banan 3.5% 100qr *48


'mimi yoqurt banan 3.5 pct 100 qr x 48'

In [616]:
df_outcoming['items'] = df_outcoming['Product or Service Name'].apply(lambda x: simple_process_item(x))
df_incoming['items'] = df_incoming['Product or Service Name'].apply(lambda x: simple_process_item(x))

In [617]:
Counter(" ".join(df_outcoming['items']).split()).most_common(50)

[('x', 10399),
 ('qr', 4951),
 ('pct', 4903),
 ('milla', 2809),
 ('1', 2582),
 ('12', 2282),
 ('200', 1683),
 ('lays', 1570),
 ('ml', 1563),
 ('24', 1557),
 ('dərman', 1449),
 ('yoqurt', 1408),
 ('48', 1385),
 ('6', 1304),
 ('100', 1192),
 ('20', 1186),
 ('president', 1168),
 ('g', 1120),
 ('kq', 1099),
 ('№', 1017),
 ('150', 1005),
 ('3.5', 1000),
 ('dsp', 944),
 ('25', 927),
 ('gr', 913),
 ('pendir', 848),
 ('xama', 823),
 ('18', 792),
 ('süd', 770),
 ('90', 731),
 ('30', 715),
 ('xrustim', 692),
 ('10', 674),
 ('300', 646),
 ('l', 637),
 ('50', 623),
 ('q', 623),
 ('8', 621),
 ('suxari', 618),
 ('1.5', 612),
 ('0.5', 593),
 ('5', 589),
 ('ayran', 551),
 ('paprika', 525),
 ('şaftalı', 506),
 ('sevimli', 500),
 ('dad', 500),
 ('3', 471),
 ('qaymaq', 466),
 ('27', 462)]

In [650]:
%%time

to_find = df_incoming['items'].to_list()[:1000]
candidates = set(df_outcoming['items'].to_list())

outcoming, incoming1, incoming2, score1, score2, score_w, score_d = [], [], [], [], [], [], []
for item in tqdm(to_find[:]):
    res1 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_token_sort_ratio,
                limit=1
    )[0]
    res2 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_ratio,
                limit=1
    )[0]
    outcoming.append(item)
    incoming1.append(res1[0])
    incoming2.append(res2[0])
    score1.append(res1[1])
    score2.append(res2[1])
    if len(res1) > 0:
        score_w.append(count_common_words(item, res1[0]))
        score_d.append(count_common_digits(item, res1[0]) / count_digit_share(res1[0]))


100%|██████████| 1000/1000 [00:13<00:00, 71.87it/s]

CPU times: user 13.9 s, sys: 27.3 ms, total: 13.9 s
Wall time: 13.9 s


In [651]:
comp = pd.DataFrame()
comp['right'] = outcoming
comp['left1'] = incoming1
comp['left2'] = incoming2
comp['score1'] = score1
comp['score2'] = score2
comp['score3'] = 2*comp['score1'] *comp['score2'] /(comp['score1'] + comp['score2'] )
comp['score_w'] = score_w
comp['score_d'] = score_d
comp['score_d'] = comp['score_d'] /10

In [652]:
# parq_write(
#     f'{os.getcwd()}/comp.parq',
#     comp,
#     compression='GZIP'
# )

In [653]:
# comp_parq = ParquetFile(f'{os.getcwd()}/comp.parq',)
# comp = comp_parq.to_pandas()

In [654]:
df_outcoming.shape, df_incoming.shape, comp.shape

((16947, 6), (937415, 6), (1000, 8))

In [655]:
sub_result = pd.concat(
    [
        df_incoming[:1000],
        comp
    ],
    axis=1,
)
sub_result.shape

(1000, 14)

In [656]:
compilation_full = sub_result.merge(df_outcoming, how='left', left_on='left1', right_on='items')
compilation_full = compilation_full.drop_duplicates(subset=['#_x']) 
compilation_full.shape

(1000, 20)

In [657]:
compilation_full = compilation_full[[
          'items_x', 'items_y',
          'Quantity/Volume_x', 'unit_x', 'Quantity/Volume_y', 'unit_y',
          'score1', 'score2', 'score3', 'score_w', 'score_d',
          'Product or Service Name_x', 'Product or Service Name_y', ]]

In [658]:
comp_true = compilation_full[
    (compilation_full.score3 > 78) 
    # & (compilation_full.score_w > 0.0)
    & (compilation_full.score_d > 0.17)
]

In [843]:
comp_doubt = compilation_full.drop(comp_true.index, axis=0)

In [660]:
comp_doubt['items_y']  = ''
comp_doubt['Quantity/Volume_y'] = 0
comp_doubt['unit_y'] = ''
# comp_doubt['Product or Service Name_y'] = ''

In [661]:
comp_true.shape[0] / comp_doubt.shape[0]

0.49700598802395207

In [844]:
comp_doubt.sample(5)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,score_d,Product or Service Name_x,Product or Service Name_y
5897,sochnaya port mango mandar 0.95 1 x 12,mirinda alma 0.5 x 12,12,adet.lt,3888,eded,68.421053,77.777778,72.800000,0.000000,0.283333,SOCHNAYA D PORT-MANGO-MANDAR 0.95LT1x12,Mirinda Alma 0.5 lt*12
5877,sochnaya dolina shaftali alma 0.95 1 x 12,mirinda alma 1 x 12,48,adet.lt,8640,eded,73.684211,77.777778,75.675676,0.285714,0.500000,SOCHNAYA DOLINA SHAFTALI-ALMA 0.95 LT 1x12,Mirinda Alma 1 lt*12
11291,superfresh milfoy 1000 gr 1 x 10,angins 10 100 ml,4,eded,2,eded,68.750000,66.666667,67.692308,0.000000,0.104000,SUPERFRESH MILFOY 1000 GR 1X10,Angins10 100 ml
8699,suxari toyuq qanadlari barbeku 90 gr 1 x 24,suxari xrustim toyuq qanadlari 90 qr x 24,10,eded,25344,eded,75.757576,73.170732,74.441687,0.750000,0.680000,SUXARI TOYUQ QANADLARI BARBEKU 90 GR 1 X 24,Suxari Xrustim Toyuq qanadlari 90 qr X24
12847,pinar sut protein kakaolu 500 ml 1 x 12,k 45 28,3,adet.ml,10,adet.lt,71.428571,77.777778,74.468085,0.000000,0.000000,PINAR SUT PROTEIN KAKAOLU 500 ML 1x12,D-k 45/d28


In [662]:
compilation_scored = pd.concat([comp_true, comp_doubt])

In [663]:
compilation_scored['unit_common'] = compilation_scored['unit_x'] + ' ' +  compilation_scored['unit_y']

In [664]:
compilation_scored['unit_common_set'] = compilation_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))

In [665]:
compilation_scored['unit_common_str'] = compilation_scored['unit_common_set'].apply(
    lambda x: str(list(x)[0]) if len(list(x)) == 0 else ' '.join(list(x))
)

In [666]:
compilation_scored['quant_in'] = compilation_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
compilation_scored['quant_out'] = compilation_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')

In [667]:
# x - incoming y - outcoming
compilation_scored[[
          'items_x', 'items_y',
          'score3', 'score1', 'score2', 'score_w', 'score_d', 'Product or Service Name_y']].sample(250)

,items_x,items_y,score3,score1,score2,score_w,score_d,Product or Service Name_y
8569,berrak jalapeno biber dilimli 370 ml 1 x 12,,73.684211,70.000000,77.777778,0.000000,0.116667,7up 1 lt*12
1367,kirlangic misir yagi 2 1 x 8,,72.727273,66.666667,80.000000,0.000000,0.146667,Dirsek 28 (1 1/8)
12889,sandora multivitamin 0.97 1 x 12,ya multivitamin 0.97 x 12,89.955556,88.000000,92.000000,0.800000,0.280000,Ya Multivitamin 0.97 lt*12
13386,burcu domates biber tomat ikisibirarada 600 gr...,,71.794872,66.666667,77.777778,0.000000,0.000000,Kif
18953,pinar ketcap plastik 400 gr 1 x 6,,72.544081,76.190476,69.230769,0.400000,0.000000,Plastik qutu
...,...,...,...,...,...,...,...,...
18325,sevimli dad xama kaunasskaya 20 pct 200 gr 1 x 12,sevimli dad xama kaunasskaya 20 pct 200 gr 1 x 12,100.000000,100.000000,100.000000,1.000000,0.487500,SEVIMLI DAD XAMA KAUNASSKAYA 20% 200 GR 1X12
18929,sonti saftali nektari 200 gr 1 x 27,,66.666667,66.666667,66.666667,0.000000,0.000000,Fan
7096,sevimli dad kesmik uzumlu 9 pct 180 gr 1 x 1,sevimli dad kesmik kaunas 0 pct 180 gr 1 x 1,85.676393,85.000000,86.363636,0.666667,0.377778,SEVIMLI DAD KESMIK KAUNAS 0% 180 GR1x1
6004,doktor milk xama 25 pct 200 qr 1 x 12,,79.160420,75.862069,82.758621,0.400000,0.089796,Milla Xama 25% 175qr *36


In [668]:
compilation_scored['items_x_numberless'] = compilation_scored['items_x'].apply(lambda x: remove_numbers(x))

In [830]:
kw_extractor1 = yake.KeywordExtractor(
    n=3,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)

kw_extractor2 = yake.KeywordExtractor(
    n=1,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=1,
    features=None
)

kw_extractor = yake.KeywordExtractor(
    n=2,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)


In [835]:
def get_2main_word(word: str):
    keywords = kw_extractor.extract_keywords(word)
    x = {kw[0]: kw[1] for kw in keywords}
    if len(x) > 0:
        return min(x, key=x.get)
    else:
        return ''

def get_1main_word(words: str):
    if len(words) > 0:
        fltr = kw_extractor.extract_keywords(words)
        fltr_dct = {kw[0]: kw[1] for kw in fltr}
        return max(fltr_dct, key=fltr_dct.get)
    else:
        return ''

In [836]:
for item in compilation_scored.sample(50)['items_x_numberless']:
    words = get_2main_word(item)
    word = get_1main_word(words)
    print(item)
    print('---')
    splitted_words = words.split()
    if len(splitted_words) == 2:
        print(' '.join(splitted_words))
        print(' '.join(splitted_words[:-1]))
        # print(' '.join(splitted_words[-1:]))
    elif len(splitted_words) == 3:
        print(' '.join(splitted_words[:-1]))
        print(splitted_words[-1])
    # print(word)
    print()

fersan uzum sirkesi
---
fersan uzum
fersan

sevimli canax pendiri
---
sevimli canax
sevimli

lays sir
---
lays sir
lays

pinar salam bitir
---
pinar salam
pinar

bebeto rainbow twist
---
bebeto rainbow
bebeto

sevimli kesmik yagli
---
sevimli kesmik
sevimli

fora siyah zeytin
---
fora siyah
fora

suxari bavarskie kolbaski
---
suxari bavarskie
suxari

icimino ciyelek
---
icimino ciyelek
icimino

sevimli kefir
---
sevimli kefir
sevimli

doktor milk xama
---
doktor milk
doktor

doktor milk kesmik
---
doktor milk
doktor

ulker puding icim
---
ulker puding
ulker

lays ridged chili
---
lays ridged
lays

beliy qorod sud
---
beliy qorod
beliy

suxari bavarskie kolbaski
---
suxari bavarskie
suxari

sevimli kesmik uzumlu
---
sevimli kesmik
sevimli

parmalat kokteyl cokolato
---
parmalat kokteyl
parmalat

pinar balik hamsi
---
pinar balik
pinar

suxari smetan
---
suxari smetan
suxari

pinar milfoy
---
pinar milfoy
pinar

milla sud
---
milla sud
milla

superfresh pizza king
---
superfresh pizza
su

In [837]:
compilation_scored['main_2word'] = compilation_scored['items_x_numberless'].apply(lambda x: get_2main_word(x))
compilation_scored['main_1word'] = compilation_scored['main_2word'].apply(lambda x: get_1main_word(x))

In [838]:
compilation_scored.groupby(by=['main_1word', 'main_2word', 'items_x', 'items_y', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}) # .sort_values(['unit_common_str'])

quant_in  \
main_1word main_2word         items_x                                            items_y                        unit_common_str             
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acılı 130 qr x 16      eded                 96.0   
                              doritos acili 74 gr 1 x 24                         doritos acılı 74 qr x 24       eded                 63.0   
adjika     qlavprodukt adjika qlavprodukt adjika domasnaya 170 qr 1 x 15                                        eded                  5.0   
                              qlavprodukt adjika kavkazskaya 170 qr 1 x 15                                      eded                  4.0   
alma       icimino alma       icimino alma erik 40 gr 1 x 48                     icimino alma ərik 6 x 40 g x 8 eded                 48.0   
...                                                                                                                                   ...   
yagli      president yagli    president yagli 140 gr 1 x 15                                                     eded                 79.0   
                              president yagli 200 gr 1 x 16                                                     eded                 20.0   
                              president yagli 280 gr 1 x 18                                                     eded                  8.0   
                              president yagli 400 gr 1 x 16                                                     eded                 16.0   
zolotoy    zolotoy petusok    zolotoy petusok naggets xirtxirt 300 gr 1 x 12 ...                                eded                  4.0   

                                                                                                                                 quant_out  
main_1word main_2word         items_x                                            items_y                        unit_common_str             
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acılı 130 qr x 16      eded              104000.0  
                              doritos acili 74 gr 1 x 24                         doritos acılı 74 qr x 24       eded              134592.0  
adjika     qlavprodukt adjika qlavprodukt adjika domasnaya 170 qr 1 x 15                                        eded                   0.0  
                              qlavprodukt adjika kavkazskaya 170 qr 1 x 15                                      eded                   0.0  
alma       icimino alma       icimino alma erik 40 gr 1 x 48                     icimino alma ərik 6 x 40 g x 8 eded                1600.0  
...                                                                                                                                    ...  
yagli      president yagli    president yagli 140 gr 1 x 15                                                     eded                   0.0  
                              president yagli 200 gr 1 x 16                                                     eded                   0.0  
                              president yagli 280 gr 1 x 18                                                     eded                   0.0  
                              president yagli 400 gr 1 x 16                                                     eded                   0.0  
zolotoy    zolotoy petusok    zolotoy petusok naggets xirtxirt 300 gr 1 x 12 ...                                eded                   0.0  

[494 rows x 2 columns]

In [842]:
compilation_scored.groupby(['main_1word']).agg({'quant_in': 'sum', 'quant_out': 'sum'})

,quant_in,quant_out
main_1word,,
acili,159.0,238592.0
adjika,9.0,0.0
alma,48.0,1600.0
angry,15.0,0.0
anl,109.0,0.0
...,...,...
xama,608.0,174612.0
xardal,21.0,0.0
xren,5.0,0.0


In [816]:
brand_agg = compilation_scored.groupby(['main_1word', 'main_2word', 'items_x', 'items_y', 'unit_common_str']).agg({'quant_in': 'sum', 'quant_out': 'sum'}).reset_index()
brand_agg.rename(columns={'quant_in': 'brand_inventory_in', 'quant_out': 'brand_inventory_out'}, inplace=True)

# Aggregate at the item_type level within each brand group
type_agg = compilation_scored.groupby(['main_1word', 'main_2word']).agg({'quant_in': 'sum', 'quant_out': 'sum'}).reset_index()

# Merge the two aggregation results
result = pd.merge(brand_agg, type_agg, on='main_1word', how='left')

In [817]:
result

,main_1word,main_2word_x,items_x,items_y,unit_common_str,brand_inventory_in,brand_inventory_out,main_2word_y,quant_in,quant_out
0,acili,doritos acili,doritos acili 130 gr 1 x 16,doritos acılı 130 qr x 16,eded,96.0,104000.0,doritos acili,159.0,238592.0
1,acili,doritos acili,doritos acili 74 gr 1 x 24,doritos acılı 74 qr x 24,eded,63.0,134592.0,doritos acili,159.0,238592.0
2,adjika,qlavprodukt adjika,qlavprodukt adjika domasnaya 170 qr 1 x 15,,eded,5.0,0.0,qlavprodukt adjika,9.0,0.0
3,adjika,qlavprodukt adjika,qlavprodukt adjika kavkazskaya 170 qr 1 x 15,,eded,4.0,0.0,qlavprodukt adjika,9.0,0.0
4,alma,icimino alma,icimino alma erik 40 gr 1 x 48,icimino alma ərik 6 x 40 g x 8,eded,48.0,1600.0,icimino alma,48.0,1600.0
...,...,...,...,...,...,...,...,...,...,...
2418,yagli,president yagli,president yagli 140 gr 1 x 15,,eded,79.0,0.0,president yagli,123.0,0.0
2419,yagli,president yagli,president yagli 200 gr 1 x 16,,eded,20.0,0.0,president yagli,123.0,0.0
2420,yagli,president yagli,president yagli 280 gr 1 x 18,,eded,8.0,0.0,president yagli,123.0,0.0
2421,yagli,president yagli,president yagli 400 gr 1 x 16,,eded,16.0,0.0,president yagli,123.0,0.0


In [821]:
result.groupby(by=['main_1word', 'main_2word_x', 'items_x', 'items_y', 'unit_common_str']).agg(
    {'quant_in':'sum','quant_out':'sum', 'brand_inventory_in': 'max', 'brand_inventory_out': 'max'}
) # .sort_values(['unit_common_str'])

quant_in  \
main_1word main_2word_x       items_x                                            items_y                        unit_common_str             
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acılı 130 qr x 16      eded                159.0   
                              doritos acili 74 gr 1 x 24                         doritos acılı 74 qr x 24       eded                159.0   
adjika     qlavprodukt adjika qlavprodukt adjika domasnaya 170 qr 1 x 15                                        eded                  9.0   
                              qlavprodukt adjika kavkazskaya 170 qr 1 x 15                                      eded                  9.0   
alma       icimino alma       icimino alma erik 40 gr 1 x 48                     icimino alma ərik 6 x 40 g x 8 eded                 48.0   
...                                                                                                                                   ...   
yagli      president yagli    president yagli 140 gr 1 x 15                                                     eded                123.0   
                              president yagli 200 gr 1 x 16                                                     eded                123.0   
                              president yagli 280 gr 1 x 18                                                     eded                123.0   
                              president yagli 400 gr 1 x 16                                                     eded                123.0   
zolotoy    zolotoy petusok    zolotoy petusok naggets xirtxirt 300 gr 1 x 12 ...                                eded                  4.0   

                                                                                                                                 quant_out  \
main_1word main_2word_x       items_x                                            items_y                        unit_common_str              
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acılı 130 qr x 16      eded              238592.0   
                              doritos acili 74 gr 1 x 24                         doritos acılı 74 qr x 24       eded              238592.0   
adjika     qlavprodukt adjika qlavprodukt adjika domasnaya 170 qr 1 x 15                                        eded                   0.0   
                              qlavprodukt adjika kavkazskaya 170 qr 1 x 15                                      eded                   0.0   
alma       icimino alma       icimino alma erik 40 gr 1 x 48                     icimino alma ərik 6 x 40 g x 8 eded                1600.0   
...                                                                                                                                    ...   
yagli      president yagli    president yagli 140 gr 1 x 15                                                     eded                   0.0   
                              president yagli 200 gr 1 x 16                                                     eded                   0.0   
                              president yagli 280 gr 1 x 18                                                     eded                   0.0   
                              president yagli 400 gr 1 x 16                                                     eded                   0.0   
zolotoy    zolotoy petusok    zolotoy petusok naggets xirtxirt 300 gr 1 x 12 ...                                eded                   0.0   

                                                                                                                                 brand_inventory_in  \
main_1word main_2word_x       items_x                                            items_y                        unit_common_str                       
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acılı 130 qr x 16      eded                           96.

In [680]:
compilation_scored[compilation_scored['items_y'] == 'sevimli dad rossiyskiy pendir 1 x 1']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word
2832,sevimli dad rossiyskiy pendir 1 x 1,sevimli dad rossiyskiy pendir 1 x 1,5804,adet.kg,2400,eded,100.000000,100.000000,100.000000,1.000000,...,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,adet.kg eded,"{kg, adet, eded}",kg adet eded,5804.0,2400.0,sevimli rossiyskiy pendir,sevimli rossiyskiy,sevimli
5691,sevimli dad canax pendiri ceki 1 x 7,sevimli dad rossiyskiy pendir 1 x 1,7,adet.kg,2400,eded,77.192982,92.307692,84.076433,0.285714,...,SEVIMLI DAD CANAX PENDIRI ( CEKI ) 1X7,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,adet.kg eded,"{kg, adet, eded}",kg adet eded,7.0,2400.0,sevimli canax pendiri,sevimli canax,sevimli
6234,sevimli dad canax pendiri ceki 1 x 7,sevimli dad rossiyskiy pendir 1 x 1,35,adet.kg,2400,eded,77.192982,92.307692,84.076433,0.285714,...,SEVIMLI DAD CANAX PENDIRI ( CEKI ) 1X7,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,adet.kg eded,"{kg, adet, eded}",kg adet eded,35.0,2400.0,sevimli canax pendiri,sevimli canax,sevimli
15966,sevimli dad canax pendiri 1 kg 1 x 10 goycay,sevimli dad rossiyskiy pendir 1 x 1,20,eded,2400,eded,67.924528,92.307692,78.260870,0.285714,...,SEVIMLI DAD CANAX PENDIRI 1 KG 1X10 (GOYCAY),SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,eded eded,{eded},eded,20.0,2400.0,sevimli canax pendiri,sevimli canax,sevimli


In [681]:
compilation_scored[compilation_scored['items_y'] == 'mia limon 1 x 12']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word


In [682]:
compilation_scored[compilation_scored['items_y'] == '7 up 1 x 12']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word


In [319]:
remove_numbers(simple_process_item('SEVIMLI.XAMA KAUNAS.20% 350GR AL 10% 200GR HDY 1X1'))

'sevimli xama kaunas'

In [801]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("azizbarank/mbert-finetuned-azerbaijani-ner", is_split_into_words=True)
model = AutoModelForTokenClassification.from_pretrained("azizbarank/mbert-finetuned-azerbaijani-ner")
nlp = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy='simple')


In [811]:
for item in compilation_scored.sample(10)['items_x']:
    ner_results = nlp(item)
    print(item)
    [print(x['word'], ' ', x['entity_group']) for x in ner_results]
    print()

doktor milk xama 25 pct 350 qr 1 x 12
doktor   LABEL_1
milk xama   LABEL_2
25 pct 350 qr 1 x 12   LABEL_0

pamuk lokum badamli narli 250 gr 1 x 12
pamuk   LABEL_5
lokum   LABEL_6
badamli narli 250 gr 1 x 12   LABEL_0

pinar salam kahvaltilik 250 gr 1 x 10
pin   LABEL_5
##ar   LABEL_3
salam   LABEL_4
kahvaltilik 250 gr 1 x 10   LABEL_0

sevimli dad kesmik yagsiz 0 pct 180 gr 1 x 1
sevimli   LABEL_3
dad kesmik yagsiz 0 pct 180 gr 1 x 1   LABEL_0

doktor milk teze sud 1.5 pct 900 qr 1 x 12
doktor   LABEL_3
milk teze sud 1. 5 pct 900 qr 1 x 12   LABEL_0

osmanli lokum badamli qarisiq 300 gr 1 x 16
osmanli   LABEL_5
lokum   LABEL_6
badamli qarisiq 300 gr 1 x 16   LABEL_0

lays ridged paprika 150 gr 1 x 18
lays   LABEL_5
ridged paprika   LABEL_6
150 gr 1 x 18   LABEL_0

bebeto spaghetti apple 80 gr 1 x 12
bebeto   LABEL_5
spaghetti   LABEL_6
apple 80 gr 1 x 12   LABEL_0

president galbani kesme mocarella 150 gr 1 x 15
president   LABEL_1
galbani   LABEL_2
kesme mocarella 150 gr 1 x 15   LABE

In [794]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = 'token-classification'
ner_results = nlp(example)
ner_results

[{'entity': 'B-ORG',
  'score': 0.9993765,
  'index': 1,
  'word': 'tok',
  'start': 0,
  'end': 3},
 {'entity': 'B-ORG',
  'score': 0.99934775,
  'index': 2,
  'word': '##en',
  'start': 3,
  'end': 5},
 {'entity': 'I-ORG',
  'score': 0.9599418,
  'index': 3,
  'word': '-',
  'start': 5,
  'end': 6},
 {'entity': 'I-ORG',
  'score': 0.986715,
  'index': 4,
  'word': 'classification',
  'start': 6,
  'end': 20}]